https://mpi4py.readthedocs.io/en/stable/tutorial.html#dynamic-process-management

In [1]:
import os 

In [2]:
import subprocess

In [3]:
master_str = """#!/usr/bin/env python
from mpi4py import MPI
import numpy
import sys

comm = MPI.COMM_SELF.Spawn(sys.executable,
                           args=['cpi.py'],
                           maxprocs=5)

N = numpy.array(100, 'i')
comm.Bcast([N, MPI.INT], root=MPI.ROOT)
PI = numpy.array(0.0, 'd')
comm.Reduce(None, [PI, MPI.DOUBLE],
            op=MPI.SUM, root=MPI.ROOT)
print(PI)

comm.Disconnect()
"""

In [4]:
cpi_str = """#!/usr/bin/env python
from mpi4py import MPI
import numpy

comm = MPI.Comm.Get_parent()
size = comm.Get_size()
rank = comm.Get_rank()

N = numpy.array(0, dtype='i')
comm.Bcast([N, MPI.INT], root=0)
h = 1.0 / N; s = 0.0
for i in range(rank, N, size):
    x = h * (i + 0.5)
    s += 4.0 / (1.0 + x**2)
PI = numpy.array(s * h, dtype='d')
comm.Reduce([PI, MPI.DOUBLE], None,
            op=MPI.SUM, root=0)

comm.Disconnect()
"""

In [5]:
if not os.path.exists('master.py'):
    with open('master.py', 'w') as f:
        f.writelines(master_str)

In [6]:
if not os.path.exists('cpi.py'):
    with open('cpi.py', 'w') as f:
        f.writelines(cpi_str)

In [7]:
subprocess.check_output('mpiexec -n 4 python master.py', shell=True, universal_newlines=True ).split()

['3.1416009869231245',
 '3.1416009869231245',
 '3.1416009869231245',
 '3.1416009869231245']